In [1]:
import random
from typing import List, Tuple


In [3]:
"""1~45에서 중복 없이 6개 뽑고 정렬"""
def generate_lotto_numbers() -> List[int]:
    nums = random.sample(range(1, 46), 6)
    nums.sort()
    return nums

In [4]:
"""당첨 번호 6개 + 보너스 번호 1개 생성"""
def generate_winning_numbers() -> Tuple[List[int], int]:
    winning = generate_lotto_numbers()
    bonus_candidates = [n for n in range(1, 46) if n not in winning]
    bonus = random.choice(bonus_candidates)
    return winning, bonus

In [5]:
"""
한 티켓과 당첨 번호/보너스 번호를 비교해서
등수 코드를 반환한다.

반환값:
    "3"  -> 3개 일치
    "4"  -> 4개 일치
    "5"  -> 5개 일치
    "5b" -> 5개 + 보너스 일치
    "6"  -> 6개 일치
    "0"  -> 나머지 (통계에 안 들어감)
"""
def check_rank(ticket: List[int], winning: List[int], bonus: int) -> str:
    hit = len(set(ticket) & set(winning))
    bonus_hit = bonus in ticket

    if hit == 6:
        return "6"
    if hit == 5 and bonus_hit:
        return "5b"
    if hit == 5:
        return "5"
    if hit == 4:
        return "4"
    if hit == 3:
        return "3"
    return "0"

In [6]:
def format_ticket(nums: List[int]) -> str:
    """로또 번호 리스트를 [a,b,c,d,e,f] 형식의 문자열로 변환"""
    return "[" + ",".join(str(n) for n in nums) + "]"

In [ ]:
"""
    하나의 학습 샘플을 생성한다.
    최종 포맷:

    money=8000
    winning=1,2,3,4,5,6
    bonus=7
    ###
    티켓수=8
    구매번호:
    [8,21,23,41,42,43]
    ...
    3개일치=1
    4개일치=0
    5개일치=0
    5개보너스일치=0
    6개일치=0
    수익률=62.5%
"""
def build_sample_one() -> str:
    # 1) 구입 금액 (1,000 ~ 20,000 사이, 1,000 단위)
    ticket_price = 1000
    ticket_count = random.randint(1, 20)  # 1~20장 사이
    money = ticket_count * ticket_price   # 항상 1000의 배수

    # 2) 사용자가 구매한 로또 티켓들
    tickets = [generate_lotto_numbers() for _ in range(ticket_count)]

    # 3) 당첨 번호 & 보너스 번호
    winning, bonus = generate_winning_numbers()

    # 4) 등수 카운트
    count_3 = 0
    count_4 = 0
    count_5 = 0
    count_5b = 0
    count_6 = 0

    for t in tickets:
        r = check_rank(t, winning, bonus)
        if r == "3":
            count_3 += 1
        elif r == "4":
            count_4 += 1
        elif r == "5":
            count_5 += 1
        elif r == "5b":
            count_5b += 1
        elif r == "6":
            count_6 += 1

    # 5) 상금 계산
    prize_3 = 5_000
    prize_4 = 50_000
    prize_5 = 1_500_000
    prize_5b = 30_000_000
    prize_6 = 2_000_000_000

    total_prize = (
        count_3 * prize_3 +
        count_4 * prize_4 +
        count_5 * prize_5 +
        count_5b * prize_5b +
        count_6 * prize_6
    )

    # 6) 수익률 계산 (money는 0 아님)
    rate = (total_prize / money) * 100
    rate_str = f"{rate:.1f}"  # 소수점 1자리

    # 7) 텍스트 조립 (우리가 정한 포맷 그대로)
    lines = []

    # IN 부분
    lines.append(f"money={money}")
    winning_str = ",".join(str(n) for n in winning)
    lines.append(f"winning={winning_str}")
    lines.append(f"bonus={bonus}")

    # 구분 토큰
    lines.append("###")

    # OUT 부분
    lines.append(f"티켓수={ticket_count}")
    lines.append("구매번호:")
    for t in tickets:
        lines.append(format_ticket(t))
    lines.append(f"3개일치={count_3}")
    lines.append(f"4개일치={count_4}")
    lines.append(f"5개일치={count_5}")
    lines.append(f"5개보너스일치={count_5b}")
    lines.append(f"6개일치={count_6}")
    lines.append(f"수익률={rate_str}%")

    return "\n".join(lines)



In [9]:
def build_dataset(n_samples: int) -> list[str]:
    """n_samples 개의 샘플 문자열 리스트 생성"""
    return [build_sample_one() for _ in range(n_samples)]

def save_dataset_to_file(texts: list[str], path: str) -> None:
    """
    각 샘플을 공백 줄로 구분해서 하나의 파일에 저장.
    (원하면 구분 없이 그냥 줄줄이 붙여도 됨)
    """
    with open(path, "w", encoding="utf-8") as f:
        for i, sample in enumerate(texts):
            f.write(sample)
            if i != len(texts) - 1:
                f.write("\n\n")  # 샘플 사이에 빈 줄 하나



In [11]:
random.seed(42)

# 원하는 크기만큼 설정
n_train = 10000
n_val = 1000
n_test = 1000

train_texts = build_dataset(n_train)
val_texts = build_dataset(n_val)
test_texts = build_dataset(n_test)

save_dataset_to_file(train_texts, "lotto_train.txt")
save_dataset_to_file(val_texts, "lotto_val.txt")
save_dataset_to_file(test_texts, "lotto_test.txt")

print("Done! lotto_train.txt / lotto_val.txt / lotto_test.txt 생성 완료")

Done! lotto_train.txt / lotto_val.txt / lotto_test.txt 생성 완료
